<a href="https://colab.research.google.com/github/srigouri/CE888/blob/master/Labs/lab6/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.set_printoptions(precision = 3)
import pandas as pd
import operator
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
ratings = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None )
print(ratings)

In [ ]:
ratings.drop(columns=[0],axis=1, inplace=True)
ratings = ratings.replace(99.00, np.NaN)
ratings.head()

In [ ]:
mean_rating = ratings.mean(axis=0)
(pd.DataFrame(mean_rating)).nlargest(10,0)
ratings.iloc[5621, 17]

arr = ratings.values
print(arr.shape)
rated = np.where(~np.isnan(arr))
print(len(rated[0]), rated[1].shape)

In [ ]:
def replace(orig, percentage=0.1):
  ndata = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  ndata[rated[0][idx], rated[1][idx]] = 99
  return ndata, (rated[0][idx], rated[1][idx])

n_arr, idx = replace(arr, 0.1)
print(arr[idx[0][0], idx[1][0]])
n_arr[idx[0][0], idx[1][0]]

In [ ]:
n_latent_factors = 2
latent_u_preferences = np.random.random((arr.shape[0], n_latent_factors))
latent_i_features = np.random.random((arr.shape[1], n_latent_factors))

def predict(u_id, i_id):
    u_preference = latent_u_preferences[u_id]
    i_preference = latent_i_features[i_id]
    return u_preference.dot(i_preference)

def train(u_id, i_id, rating, alpha=0.0001):
    predicted_r = predict(u_id, i_id)
    error =  predicted_r - rating
    upref_values = latent_u_preferences[u_id]
    latent_u_preferences[u_id] -= alpha * error * latent_i_features[i_id]
    latent_i_features[i_id] -= alpha * error * upref_values
    return error

def sgd(iterations):
    mse_history = []
    for iter in range(iterations):
        error = []
        for u_id in range(latent_u_preferences.shape[0]):
            for i_id in range(latent_i_features.shape[0]):
                rating = arr[u_id, i_id]
                if (~np.isnan(rating)) and rating!=99 :
                    err = train(u_id, i_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iter % 10) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iter, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
predictions = latent_u_preferences.dot(latent_i_features.T)
predictions
values = [zip(arr[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = ratings.columns
comparison_data

In [ ]:
num_iter = 100
hist = sgd(num_iter)
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/100:	MSE=26.152446


**Movie**

In [ ]:
df1 = pd.ExcelFile('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true')
df_i = pd.read_excel('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true',sheet_name='Items',index_col=None)
df_u = pd.read_excel('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true',sheet_name='Users', index_col=None)

In [ ]:
df_i.head()

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566


In [ ]:
num_factors = df_u.shape[1]-1
u_pref = df_u[df_u['User'] == 3519].iloc[:, -num_factors:].values[0]
i_feat = df_i.iloc[:, -num_factors:].values
pref = i_feat.dot(u_pref)
pref

array([ 0.485, -0.082,  0.02 ,  0.14 , -0.177,  0.113,  0.084,  0.067,
        0.093,  0.258, -0.045,  0.178,  0.034,  0.017,  0.046,  0.021,
        0.04 ,  0.121,  0.062,  0.08 ,  0.116,  0.041, -0.065, -0.006,
       -0.086,  0.199, -0.036,  0.198,  0.072,  0.073,  0.061,  0.159,
        0.21 ,  0.113,  0.087,  0.016,  0.063, -0.128,  0.12 , -0.03 ,
        0.011,  0.035, -0.008, -0.019,  0.119,  0.027, -0.095,  0.008,
        0.156, -0.148,  0.034, -0.334, -0.178,  0.15 , -0.058,  0.041,
       -0.083, -0.06 , -0.071,  0.246,  0.028, -0.11 ,  0.067, -0.185,
        0.024, -0.222, -0.261, -0.042, -0.512,  0.07 , -0.051, -0.008,
       -0.087,  0.014, -0.039, -0.204, -0.141,  0.503,  0.449, -0.022,
        0.058, -0.116, -0.017,  0.005, -0.016, -0.533, -0.069,  0.074,
       -0.041, -0.076, -0.545, -0.058,  0.015, -0.097, -0.079, -0.073,
       -0.017, -0.074, -0.057, -0.061])

In [ ]:
df_fact = df_i[['Movie ID','Factor2']]
df_fact.sort_values('Factor2')

,Movie ID,Factor2
16,122,-2.526300
15,121,-2.512933
14,120,-2.510061
51,604,-2.004450
79,1894,-1.711204
...,...,...
6,38,1.771799
32,275,1.777301
5,24,1.997440
59,680,2.085395


In [ ]:
top_val = pd.DataFrame(pref).nlargest(3, 0)
top_indc = top_val.index
list(top_indc)

[77, 0, 78]

In [ ]:
df_fact = df_i[['Movie ID','Factor1']]

val = df_fact[df_fact['Movie ID']==36955]['Factor1'].values[0]

round(val,3)

1.734

In [ ]:
df_i.iloc[list(top_indc)]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
77,1891,Star Wars: Episode V - The Empire Strikes Back...,-1.732354,-1.031041,2.221631,0.310886,-0.900830,2.306469,-1.847214,0.231435,-0.096332,-0.673135,-1.010806,0.657123,0.141075,0.998841,-0.115583
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
78,1892,Star Wars: Episode VI - Return of the Jedi (1983),-1.353408,-0.965083,2.099767,0.143335,-0.538405,2.495382,-1.925854,0.264722,-0.467617,-0.553613,-0.912548,0.737907,0.302659,0.969395,-0.102475
